# Topic Modelling
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [4]:
import json
from pathlib import Path
import glob
from nltk.corpus import stopwords

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [5]:
stop_words = stopwords.words('german')

## Load Data

In [6]:
records = []
for f in glob.glob(str(Path('../export/tags/*.json'))):
    records = records + json.load(open(f, 'r', encoding='utf-8'))

print(len(records))

46521


In [7]:
# Members only, no Presidents
r_members = list(filter(lambda x: x['ismember'] == True, records))
r_members = list(filter(lambda x: x['funktion'] not in ['Präsidium', '2. Vizepräsidium', '1. Vizepräsidium'], r_members))
print(len(r_members))

34497


## Prepare Data

In [8]:
# Remove Stopwords and only keep noun, adj, vb and adv

keep = ['NN', 'NE',
    'ADJA', 'ADJD',
    'VMFIN', 'VAFIN', 'VVFIN', 'VAIMP', 'VVIMP', 'VVINF', 'VAINF', 'VMINF', 'VVIZU', 'VVPP', 'VMPP', 'VAPP',
    'ADV']

for votum in r_members:
    # Reversed Loop, we will remove items from the list during loop!
    for i in reversed(range(0, len(votum['tags']))):
        tag = votum['tags'][i]
        if tag[0].lower() in stop_words:

            # Stopword found. Remove
            votum['tags'].remove(tag)

        elif tag[2] not in keep:
            
            # Not tag we need. Remove
            votum['tags'].remove(tag)

In [9]:
# Stick the words together to get a nice list for our corpora and split by sex
data_m = []
data_w = []
for votum in r_members:
    if votum['geschlecht'] == 'm':
        data_m.append(list(map(lambda tag: tag[0], votum['tags'])))
    else:
        data_w.append(list(map(lambda tag: tag[0], votum['tags'])))

# Build Dictionary
id2word_m = corpora.Dictionary(data_m)
id2word_w = corpora.Dictionary(data_w)

# Term Document Frequency
corpus_m = [id2word_m.doc2bow(text) for text in data_m]
corpus_w = [id2word_w.doc2bow(text) for text in data_w]

## Go LDA

In [7]:
lda_model_m = gensim.models.ldamodel.LdaModel(corpus=corpus_m, id2word=id2word_m, num_topics=20, random_state=100, update_every=1,
                                            chunksize=100, passes=10, alpha='auto', per_word_topics=True)

In [9]:
lda_model_w = gensim.models.ldamodel.LdaModel(corpus=corpus_w, id2word=id2word_w, num_topics=20, random_state=100, update_every=1,
                                            chunksize=100, passes=10, alpha='auto', per_word_topics=True)

In [12]:
# Store them
lda_model_m.save(str(Path('../export/models/lda_m.model')))
lda_model_w.save(str(Path('../export/models/lda_w.model')))

## Load models, if needed

In [3]:
# If needed...
lda_model_m = gensim.models.LdaModel.load(str(Path('../export/models/lda_m.model')))
lda_model_w = gensim.models.LdaModel.load(str(Path('../export/models/lda_w.model')))

## Men

In [10]:
pyLDAvis.enable_notebook()

In [11]:
lda_viz = gensimvis.prepare(lda_model_m, corpus_m, id2word_m)
lda_viz

d:\Entwicklung\_Projects\nzz_zh_kantonsrat_genderstudy\env\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


## Women

In [15]:
lda_viz = gensimvis.prepare(lda_model_w, corpus_w, id2word_w)
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.424188 -0.161746       1        1  40.779796
8     -0.368951  0.079361       2        1  24.849308
10    -0.355503 -0.241101       3        1  15.702385
6     -0.186864  0.245982       4        1   5.314554
9     -0.169823  0.275142       5        1   4.474654
1     -0.048192  0.054865       6        1   2.222229
14     0.067953 -0.009023       7        1   1.029309
3      0.076109 -0.011576       8        1   0.839129
19     0.082447 -0.013371       9        1   0.750726
13     0.079405 -0.012529      10        1   0.664869
4      0.089064 -0.017992      11        1   0.619963
11     0.104405 -0.018606      12        1   0.614473
17     0.108729 -0.019407      13        1   0.568556
5      0.114404 -0.019629      14        1   0.472419
0      0.102195 -0.017848      15        1   0.423445
7      0.136905 -0.022703      16        1   0.245957
12     0.135288 -0.022619      17        1   0.235433
16     0.145441 -0.022829      18        1   0.165118
15     0.155445 -0.022201      19        1   0.014748
18     0.155731 -0.022170      20        1   0.012930, topic_info=                      Term          Freq         Total Category  logprob  \
13                  Kanton  30998.000000  30998.000000  Default   30.000   
26                  Zürich  28980.000000  28980.000000  Default   29.000   
357                     ja  17895.000000  17895.000000  Default   28.000   
1747               Vorlage  10322.000000  10322.000000  Default   27.000   
1700            Kommission  10036.000000  10036.000000  Default   26.000   
...                    ...           ...           ...      ...      ...   
6739              passiert      0.002401      0.942220  Topic20  -12.028   
6331        schlussendlich      0.002401      0.942196  Topic20  -12.028   
12699            Richtplan      0.002401      0.942215  Topic20  -12.028   
18747  Eigentümerstrategie      0.002401      0.942209  Topic20  -12.028   
9929                Bauten      0.002401      0.942196  Topic20  -12.028   

       loglift  
13     30.0000  
26     29.0000  
357    28.0000  
1747   27.0000  
1700   26.0000  
...        ...  
6739    2.9810  
6331    2.9811  
12699   2.9810  
18747   2.9810  
9929    2.9810  

[782 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
1216      5  0.998792            1.
4116     16  0.988942           17.
3261      8  0.996202           18.
5082      8  0.996195         Abbau
2509      4  0.999514     Ablehnung
...     ...       ...           ...
1032      8  0.996178        ändert
2213      1  0.766681  öffentlichen
2213      2  0.179834  öffentlichen
2213     11  0.053216  öffentlichen
1616      6  0.998569     überlegen

[684 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 9, 11, 7, 10, 2, 15, 4, 20, 14, 5, 12, 18, 6, 1, 8, 13, 17, 16, 19])